# Analyzing CIA Factbook Data Using SQL

## Introduction

We'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
- population — the global population.
- population_growth — the annual population growth rate, as a percentage.
- area — the total land and water area.

Now we are going to load extention and conect with the database file

In [3]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


As we see our database file contains two tables, we'll work on table **facts**

First explor the table by querying for the first five rows of the facts table in the database.
and it look like this:



In [5]:
%%sql
SELECT * 
FROM facts
  LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


**Here are the descriptions for the columns:**

   - **name** — the name of the country.
   - **area**— the country's total area (both land and water).
   - **area_land** — the country's land area in square kilometers.
   - **area_water** — the country's waterarea in square kilometers.
   - **population** — the country's population.
   - **population_growth**— the country's population growth as a percentage.
   - **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.
   - **death_rate** — the country's death rate, or the number of death per year per 1,000 people.

**Now we start by calculating some summary statistics and look for any outlier countries.**

In [6]:
%%sql
SELECT MIN(population) AS "Minimum_population",
       MAX(population) AS "Maximum_population",
       MIN(population_growth) AS "Minimum_population_growth",
        MAX(population_growth) AS "Maximum_population_growth"
    FROM facts
       

 * sqlite:///factbook.db
Done.


Minimum_population,Maximum_population,Minimum_population_growth,Maximum_population_growth
0,7256490011,0.0,4.02


## Exploring Outliers

We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)
- Let's use subqueries to zoom in on just these countries without using the specific values.

Let's use subqueries to zoom in on just these countries without using the specific values.

**A query that returns the countries with the minimum population.**


In [29]:
%%sql
SELECT * 
 FROM facts
WHERE population == (SELECT MIN(population)
                      FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


**A query that returns the countries with the maximum population.**


In [30]:
%%sql
SELECT * 
    FROM facts
    WHERE population == (SELECT MAX(population) 
                          FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


## Exploring Average Population and Area

It seems like the table contains **a row for the whole world, which explains the population of over 7.2 billion**.


It also seems like the table contains **a row for Antarctica, which explains the population of 0**. This seems to match the CIA Factbook page for Antarctica: "no indigenous inhabitants, but there are both permanent and summer-only staffed research stations note: 53 countries have signed the 1959 Antarctic Treaty; ..." 

**Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.**


Recompute the summary statistics you found earlier while excluding the row for the whole world. Include the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth


In [44]:
%%sql
SELECT MIN(population) AS "Minimum_population",
       MAX(population) AS "Maximum_population",
       MIN(population_growth) AS "Minimum_population_growth",
        MAX(population_growth) AS "Maximum_population_growth"
    FROM facts
WHERE name NOT IN("Antarctica", "World");

 * sqlite:///factbook.db
Done.


Minimum_population,Maximum_population,Minimum_population_growth,Maximum_population_growth
48,1367485388,0.0,4.02


We'r going to calculate the average value for the following columns:
   - population
   - area 

In [60]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
    FROM facts
WHERE name NOT IN("Antarctica", "World");

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32377011.0125,555093.546184739


##  Finding Densely Populated Countries

 We'll identify countries that have the following:

- Above-average values for population.
- Below-average values for area.

In [46]:
%%sql
SELECT *
 FROM facts
WHERE population > (SELECT AVG(population) FROM facts WHERE name NOT IN("Antarctica", "World")) 
        AND
      area < (SELECT AVG(area) FROM facts WHERE name NOT IN("Antarctica", "World"))
        AND
        name NOT IN("Antarctica", "World")
;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


**There 14 countries that has the above-average values for population and below-average values for area:**
1. Bangladesh
2. Germany
3. Iraq
4. Italy
5. Japan
6. Korea, South
7. Morocco
8. Philippines
9. Poland
10. Spain
11. Thailand
12. Uganda
13. United Kingdom
14. Vietnam

## Exploring  population characteristics for the counrties

**country has the highest population rate**

In [59]:
%%sql
SELECT name AS country, 
       MAX(population)
    FROM facts
    WHERE name NOT IN("Antarctica", "World")

 * sqlite:///factbook.db
Done.


country,MAX(population)
China,1367485388


**country has the highest growth rate**

In [58]:
%%sql
SELECT name AS country,
       MAX(population_growth) AS "highest_growth_rate"
    
    FROM facts;


 * sqlite:///factbook.db
Done.


country,highest_growth_rate
South Sudan,4.02


**countries have more water than land**

In [67]:
%%sql
SELECT name AS country,
       area_water,
       area_land
FROM facts
WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


country,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


**country have the highest ratios of water to land**

In [65]:
%%sql
SELECT name AS country,
       MAX(area_water) AS "highest_ratios_water"
FROM facts

      
       

 * sqlite:///factbook.db
Done.


country,highest_ratios_water
Canada,891163


**country will add the most people to their populations next year**

In [64]:
%%sql
SELECT name,
       MAX((population)*(1+population_growth/100)) 
  FROM facts
WHERE name NOT IN("Antarctica", "World")

 * sqlite:///factbook.db
Done.


name,MAX((population)*(1+population_growth/100))
China,1373639072.2459998


**country have a higher death rate than birth rate**

In [54]:
%%sql
SELECT name,
       MAX(death_rate) AS "higher_death_rate"
FROM facts
WHERE death_rate > birth_rate


 * sqlite:///factbook.db
Done.


name,higher_death_rate
Ukraine,14.46


**countries have the highest population/area ratio**

In [55]:
%%sql
SELECT name,
       population / area AS Population_density
FROM facts
WHERE name NOT IN("Antarctica", "World")
ORDER BY Population_density DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,Population_density
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


## Conclusion

After analysis we have the conclusion and summary we pointed it on:

1. **The most population country in the word is China same country will add the most people to their populations next year**

2. **The most population growth in the word is South Sudan**

3. **The most country have a higher death rate than birth rate is Ukraine**

4. **The country that have the highest ratios of water to land is Canada**



5. **We found that there are two countries that have much bigger water area than land:**
    - British Indian Ocean Territory
    - Virgin Islands 


And we should noticed that caclulate the Population density by compute the population/area is more accurate than by just take the consider of high of population and small area
